## convert ground truth url to title/ year/ genre

In [30]:
import json,re
from datetime import datetime

In [37]:
truth_path ="/Users/oscar/Dropbox/1) Schoolwork/DSCI 558 Building Knowledge Graphs/Homework/HW3/Instruction/imdb_afi_el.dev.json"

with open(truth_path,'r') as f:
    truch_dict_list = json.loads(f.read())

In [156]:
afi_path = "./Task1 Entity Resolution/afi.jl"
imdb_path = "./Task1 Entity Resolution/imdb.jl"
afi_dict = dict()
imdb_dict = dict()

# afi
count = 0
for line in open(afi_path,'r').read().split('\n'):
    afi = json.loads(line)
    url = afi.get('url',None)
    title = afi.get('title',None)
    release_date = afi.get('release_date',None)
    genre = afi.get('genre',None)
    
    afi_year = 0
    if release_date == None:
        afi_year = None     
    elif re.match(r"\d{1,2}[, ]\w{3,9}[, ]{1,2}\d{4}",release_date):
#         afi_year = datetime.strptime(release_date,'%d %b, %Y').strftime("%Y")
        afi_year = release_date[-4:]
    elif re.match(r"\d\d/\d\d/\d\d\d\d",release_date):
        afi_year = release_date[-4:]
    elif re.match(r"\w{3,9}[, ]{1,2}\d{4}",release_date):
        afi_year = release_date[-4:]
    elif re.match(r"\d{4}",release_date):
        afi_year = release_date
    else:
        print(count+1,title)

    afi_dict[url] = {'title':title,'release_date':afi_year,'genre':genre}
    
    count += 1
# imdb
for line in open(imdb_path,'r').read().split('\n'):
    imdb = json.loads(line)
    url = imdb.get('url',None)
    name = imdb.get('name',None)
    year = imdb.get('year',None)
    genre = imdb.get('genre',None)
        
    imdb_dict[url] = {'name':name,'year':year,'genre':genre}

In [157]:
truth_list = []

for truth in truch_dict_list:
    imdb_movie = truth['imdb_movie']
    afi_movie = truth['afi_movie']
    
    if imdb_movie == None:
        imdb = None
    else:    
        imdb = {
            'url':imdb_movie, 
            'name':imdb_dict[imdb_movie]['name'], 
            'year':imdb_dict[imdb_movie]['year'],
            'genre':imdb_dict[imdb_movie]['genre']
        }
    
    if afi_movie == None:
        afi = None
    else:
        afi = {
            'url': afi_movie,
            'title':afi_dict[afi_movie]['title'],
            'release_date': afi_dict[afi_movie]['release_date'],
            'genre':afi_dict[afi_movie]['genre']
        }
    truth_list.append({"imdb":imdb,"afi":afi})

In [100]:
len(truth_list)

50

In [97]:
# import Levenshtein

from strsimpy.levenshtein import Levenshtein
levenshtein = Levenshtein()

from strsimpy.cosine import Cosine
cosine = Cosine(10)

from strsimpy.metric_lcs import MetricLCS

metric_lcs = MetricLCS()

In [187]:
print(1- metric_lcs.distance("Toy Story 3","Toy Story"))

0.8181818181818182


In [178]:
for truth in truth_list:
    if truth['imdb'] == None or truth['afi'] == None:
        continue
    imdb_name = truth['imdb']['name'].strip()
    afi_title = truth['afi']['title'].strip()
    similarity = 1 - metric_lcs.distance(imdb_name.lower(),afi_title.lower())
#     print(similarity)
    print(imdb_name,'|',afi_title)
    imdb_genre = truth['imdb']['genre']
    afi_genre = truth['afi']['genre']
#     print(imdb_genre,'｜',afi_genre)
    
    imdb_year = truth['imdb']['year']
    afi_year = truth['afi']['release_date']
#     print(imdb_year,afi_year)
    
#     print("URL\tIMDB:{}\tAFI:{}".format(truth['imdb']['url'],truth['afi']['url']))
#     print("TITLE\tIMDB:{}\tAFI:{}\tEdit:{}".format(imdb_name,afi_title,distance))
#     print("YEAR\tIMDB:{}\tAFI:{}\t".format(truth['imdb']['year'],truth['afi']['release_date']))

Intolerance: Love's Struggle Throughout the Ages | Intolerance
King Kong | KING KONG
Miracle on 34th Street | MIZACLE ON 3ATH STREET
Sunset Blvd. | SUNSET BLVD.
A Streetcar Named Desire | STREETCAR NAMED DESIRE , A
High Noon | HIGH NOON
Sweet Smell of Success | Sweet Smell of Success
Doctor Zhivago | DOCIOR ZHIVAGO
2001: A Space Odyssey | 2001: A SPACE ODYSSEY
Minority Report | Minority Report
Argo | Argo
Inception | Inception
Whiplash | Whiplash
Rocky | RVCKY
Apocalypse Now | APOCALYPSERNOW
Blue Velvet | BLUE VOLVET
Die Hard | DIE HARD


In [216]:
import rltk

tokenizer = rltk.NGramTokenizer()
tokenizer.basic("The Matrix",2)

{'#M', 'Ma', 'Th', 'at', 'e#', 'he', 'ix', 'ri', 'tr'}

In [183]:
export = [
{"imdb_movie": "https://www.imdb.com/title/tt0033467/", "afi_movie": "https://catalog.afi.com/#dc440a1a7fa4a6bd30f183eded493ef2" }, 
{ "imdb_movie": "https://www.imdb.com/title/tt0108052/", "afi_movie": "https://catalog.afi.com/#642a1d0b14872b56d8fde9228170da6f" }
]
with open("testoutput.json",'w') as f:
    json.dump(export,f)